In [1]:
!pip install -q beautifulsoup4 google-generativeai google-auth google-auth-oauthlib google-auth-httplib2 google-auth-oauthlib gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai
import google.auth
from google.colab import userdata
from google.colab import auth

# Model config

GEMINI = "gemini-2.5-pro-exp-03-25"

gemini_token = userdata.get("GOOGLE_API_KEY")

# Authenticate (Ensure you're logged in with Google Cloud credentials)
from google.colab import auth
auth.authenticate_user()

# Configure Gemini API
genai.configure(api_key=gemini_token)

In [3]:
# Define headers to mimic a browser request
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    )
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links.
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")

        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            # Remove irrelevant elements
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        # Extract and convert relative links to absolute URLs
        self.links = [
            urljoin(self.url, link.get("href"))  # Convert to absolute URL
            for link in soup.find_all("a") if link.get("href") and link.get("href") != "/"
        ]

    def get_contents(self):
        """
        Returns formatted webpage details including title and text content.
        """
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n\n"

In [4]:
# website  = Website("https://ikano.co.uk/")
# website.links

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages. \
\n Please always respond only in valid JSON format.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(website):
    return (
        f"Here is the list of links on the website of {website.url} - "
        "please decide which of these are relevant web links for a brochure about the company, "
        "respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\n"
        "Links (some might be relative links):\n"
        + "\n".join(website.links)
    )


In [7]:
# get_links_user_prompt(website)

In [8]:
def add_message(new_message, contents=None):
    """
    Appends a new message to the contents list.

    Args:
        contents (list): The existing list of messages.
        new_message (str): The new message to be added.

    Returns:
        list: Updated contents list.
    """
    if contents is None:
        contents = []  # Initialize an empty list if None is provided

    if not isinstance(contents, list):
        raise ValueError("Contents should be a list." , new_message)
    if not isinstance(new_message, str):
        raise ValueError("New message should be a string.", new_message)

    contents.append(new_message)
    return contents


In [9]:
def generate_text(prompt, stream=False):
    model = genai.GenerativeModel(GEMINI)
    response = model.generate_content(prompt, stream=stream)
    return response.text

In [10]:
import json
import re

def get_links(url):
    website = Website(url)

    message = add_message(link_system_prompt);
    message = add_message(get_links_user_prompt(website), message);

    # Get the model pipeline
    output = generate_text(message)

    # Extract and parse JSON using regex and json.loads
    try:
        json_match = re.search(r'\{.*\}', output, re.DOTALL)
        if json_match:
            extracted_json = json.loads(json_match.group())
            return extracted_json
        else:
            raise ValueError("No valid JSON found in response.")
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing JSON: {e}")
        return {"error": "Invalid JSON response from model"}


In [11]:
# print(get_links("https://ikano.co.uk/"))

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)

    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
# print(get_all_details("https://ikano.co.uk/"))

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
# get_brochure_user_prompt("Ikano", "https://ikano.co.uk/")

In [17]:
from IPython.display import Markdown, display

def create_brochure(company_name, url):
    # Construct the prompt
    prompt = add_message(system_prompt)
    prompt = add_message(get_brochure_user_prompt(company_name, url), prompt)

    # Generate output in streaming format
    model = genai.GenerativeModel(GEMINI)

    response = model.generate_content(prompt, stream=True)

    # Stream and display the response as Markdown
    markdown_output = ""
    for chunk in response:
        if chunk.text:
            markdown_output += chunk.text
            display(Markdown(chunk.text))  # Stream Markdown display

    print("Output create_brochure: ", markdown_output)
    return markdown_output  # Return the full streamed response


In [18]:
# create_brochure("Ikano", "https://ikano.co.uk/")

In [ ]:
# Gradio Interface
import gradio as gr

def on_select(selected):
    if selected:
        return create_brochure("Ikano", selected.strip())
    return "Select valid URL"

options = [
    "https://bank.ikano", "https://ikanobank.at", "https://ikanobank.dk", "https://ikanobank.fi",
    "https://ikanobank.de", "https://ikanobank.no", "https://ikanobank.pl", "https://ikanobank.se",
    "https://ikano.co.uk", "https://open.bank.ikano"
]

with gr.Blocks() as demo:
    dropdown = gr.Dropdown(choices=options, label="Select a Banking Portal")
    output = gr.Markdown()
    dropdown.change(on_select, inputs=[dropdown], outputs=[output])

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dab825239c6a56dd64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Okay,

 here is a short brochure about Ikano Bank based on the provided website information

, formatted in Markdown.

---

## Ikano Bank: Simple &

 Fair Financial Solutions

**(Rooted in IKEA Values)**

Welcome to Ikano

...

 Finland
Phone: 010 7735 71

8
Business ID (Y-tunnus): 2275

700-4

*Copyright © 2025 Ikano Bank*



---

Output create_brochure:  Okay, here is a short brochure about Ikano Bank based on the provided webpage content, formatted in Markdown.

---

# Ikano Bank: Financing for a Better Life

**(A Brochure for Prospective Customers, Investors, and Recruits)**

---

## About Ikano Bank

Welcome to Ikano Bank! We are part of the Ingka Group and operate across multiple European countries, including Sweden, Denmark, Norway, the UK, Germany, Austria, Poland, and Finland (established 2009).

Our vision is **to create possibilities for a better life**. We achieve this by offering simple, fair, and affordable financial services designed to improve the financial well-being of individuals and businesses. Our roots are in Sweden, with our head office in Malmö and origins in Älmhult.

---

## Our Services

We aim to make finance accessible and support responsible living through our offerings:

*   **For Businesses:** We provide flexible and competitive **leasing financing** solutions to help companies acq